In [1]:
import pandas as pd
messages =pd.read_csv('smsspamcollection/SMSSpamCollection.txt', sep = '\t', names= ['label','message'])

In [2]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
messages.shape

(5572, 2)

In [5]:
import re
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Suyash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()


In [8]:
corpus = []
for i in range(0,len(messages)):
    review = re.sub('^[a-zA-Z]',' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

In [9]:
corpus

['jurong point, crazy.. avail bugi n great world la e buffet... cine got amor wat...',
 'k lar... joke wif u oni...',
 "ree entri 2 wkli comp win fa cup final tkt 21st may 2005. text fa 87121 receiv entri question(std txt rate)t&c' appli 08452810075over18'",
 'dun say earli hor... u c alreadi say...',
 'ah think goe usf, live around though',
 "reemsg hey darl 3 week' word back! i'd like fun still? tb ok! xxx std chg send, £1.50 rcv",
 'ven brother like speak me. treat like aid patent.',
 "per request 'mell mell (oru minnaminungint nurungu vettam)' set callertun callers. press *9 copi friend callertun",
 'inner!! valu network custom select receivea £900 prize reward! claim call 09061701461. claim code kl341. valid 12 hour only.',
 'ad mobil 11 month more? u r entitl updat latest colour mobil camera free! call mobil updat co free 08002986030',
 "'m gonna home soon want talk stuff anymor tonight, k? i'v cri enough today.",
 'ix chanc win cash! 100 20,000 pound txt> csh11 send 87575. cost 

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 2500,binary= True,ngram_range=(2,2))
X = cv.fit_transform(corpus).toarray()

In [19]:
y = pd.get_dummies(messages['label'])
y = y.iloc[:,1].values

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train , y_test = train_test_split(X,y,test_size=0.2, random_state= 0)

In [22]:
from sklearn.naive_bayes import  MultinomialNB
spam_detection_model = MultinomialNB().fit(X_train,y_train)

In [23]:
y_pred = spam_detection_model.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score, classification_report
ac = accuracy_score(y_test, y_pred)
ac

0.9721973094170404

In [25]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.97      1.00      0.98       955
        True       1.00      0.81      0.89       160

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [26]:
#Creating TFIDF model


from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features= 2500, ngram_range=(1,2))
X = tv.fit_transform(corpus).toarray()

In [27]:
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [28]:
spam_detection_model = MultinomialNB().fit(X_train,y_train)

In [29]:
y_pred = spam_detection_model.predict(X_test)

In [30]:
accuracy_score(y_test,y_pred)

0.9838565022421525

WORD2VEC IMPLEMENTATION


In [152]:
import gensim
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [153]:
corpus = []
for i in range(0,len(messages)):
    review = re.sub('^[a-zA-Z]',' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

In [154]:
corpus

['jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...',
 'k lar... joking wif u oni...',
 "ree entry 2 wkly comp win fa cup final tkts 21st may 2005. text fa 87121 receive entry question(std txt rate)t&c's apply 08452810075over18's",
 'dun say early hor... u c already say...',
 'ah think go usf, life around though',
 "reemsg hey darling 3 week's word back! i'd like fun still? tb ok! xxx std chgs send, £1.50 rcv",
 'ven brother like speak me. treat like aid patent.',
 "per request 'melle melle (oru minnaminunginte nurungu vettam)' set callertune callers. press *9 copy friend callertune",
 'inner!! valued network customer selected receivea £900 prize reward! claim call 09061701461. claim code kl341. valid 12 hour only.',
 'ad mobile 11 month more? u r entitled update latest colour mobile camera free! call mobile update co free 08002986030',
 "'m gonna home soon want talk stuff anymore tonight, k? i've cried enough today.",
 'ix chance win cash! 100 20

In [155]:
len(corpus)

5572

In [156]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [220]:
words = []
for sent in corpus:
    sent_tokens = simple_preprocess(sent)  # Process the sentence as a whole
    words.append(sent_tokens)

In [221]:
words[5532]

['appened', 'adventuring']

In [222]:
len(words)

5572

In [240]:
model = gensim.models.Word2Vec(words , window= 5, min_count=1)

In [241]:
model.wv.index_to_key

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'day',
 'know',
 'll',
 'now',
 'free',
 'like',
 'time',
 'you',
 'come',
 'got',
 'love',
 'text',
 'want',
 'good',
 'me',
 'send',
 'it',
 'need',
 'txt',
 'one',
 'going',
 'home',
 'stop',
 'lor',
 'today',
 'see',
 'ok',
 'mobile',
 'take',
 'back',
 'da',
 'still',
 'reply',
 'think',
 'ou',
 'week',
 'phone',
 'later',
 'new',
 'tell',
 'dont',
 'co',
 'make',
 'msg',
 'hat',
 'much',
 'claim',
 'say',
 'night',
 'please',
 'pls',
 'number',
 'give',
 'work',
 'message',
 'thing',
 'min',
 'way',
 'hope',
 'www',
 'orry',
 'there',
 'happy',
 'great',
 'prize',
 'ey',
 'no',
 'that',
 'right',
 'tomorrow',
 'already',
 'friend',
 'said',
 'ow',
 'ask',
 'cash',
 'amp',
 'let',
 'wat',
 'dear',
 've',
 'find',
 'tone',
 'meet',
 'really',
 'well',
 'can',
 'life',
 'miss',
 'morning',
 'win',
 'uk',
 'com',
 'care',
 'im',
 'last',
 'service',
 'nokia',
 'would',
 'feel',
 'contact',
 'every',
 'year',
 'babe',
 'something',
 'keep'

In [242]:
model.corpus_count

5572

In [243]:
model.epochs

5

In [244]:
model.wv.similar_by_word('hope')

[('like', 0.999697208404541),
 ('know', 0.999679684638977),
 ('good', 0.999675989151001),
 ('get', 0.9996657967567444),
 ('day', 0.9996615648269653),
 ('give', 0.9996553063392639),
 ('one', 0.9996539354324341),
 ('want', 0.9996395111083984),
 ('ur', 0.9996355175971985),
 ('you', 0.9996317625045776)]

In [229]:
import numpy as np 
def avg_word2vec(doc):
    x = np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
    
    return x

In [230]:
from tqdm import tqdm

In [231]:
type(model.wv.index_to_key)

list

In [232]:
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))


  0%|          | 0/5572 [00:00<?, ?it/s]c:\Users\Suyash\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Suyash\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5572/5572 [00:00<00:00, 12397.33it/s]


In [233]:
type(X)

list

In [110]:
words[0]

['jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [246]:
X_new = np.array(X, dtype= object)